In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Подключаем датасет

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cars.csv')
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   ... feature_1  feature_2 feature_3 feature_4  feature_5  feature_6  \
0  ...      True       True      True

### Удаление ненужных и пустых столбцов

In [ ]:
columns_to_drop = ['engine_has_gas','body_type','state','drivetrain','color','model_name','engine_fuel','manufacturer_name','engine_capacity','location_region','transmission','engine_type','feature_0','feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed']
df = df.drop(columns=columns_to_drop, axis=1)
df.dropna(inplace=True)

print(df.head(5))

   odometer_value  year_produced  has_warranty  price_usd  is_exchangeable  \
0          190000           2010         False   10900.00            False   
1          290000           2002         False    5000.00             True   
2          402000           2001         False    2800.00             True   
3           10000           1999         False    9999.00             True   
4          280000           2001         False    2134.11             True   

   number_of_photos  up_counter  
0                 9          13  
1                12          54  
2                 4          72  
3                 9          42  
4                14           7  


### Кодирование текстовых значений

In [ ]:
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

print(df.head(5))

   odometer_value  year_produced  has_warranty  price_usd  is_exchangeable  \
0          190000           2010         False   10900.00            False   
1          290000           2002         False    5000.00             True   
2          402000           2001         False    2800.00             True   
3           10000           1999         False    9999.00             True   
4          280000           2001         False    2134.11             True   

   number_of_photos  up_counter  
0                 9          13  
1                12          54  
2                 4          72  
3                 9          42  
4                14           7  


Предполагаем, что столбец is_exchangeable - это метки классов

In [ ]:
X = df.drop(columns=['is_exchangeable'])
y = df['is_exchangeable']

Разбиваем данные на обучающий и тестовый наборы

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
svm_sgdc = SGDClassifier()
svm_sgdc.fit(X_train, y_train)

SGDClassifier()

In [ ]:
svm_sgdc_pred = svm_sgdc.predict(X_test)

print(f'Сравнение метрик: \n{classification_report(y_test, svm_sgdc_pred)}\n')
print(f'Матрица ошибок: \n{confusion_matrix(y_test, svm_sgdc_pred)}\n')

Сравнение метрик: 
              precision    recall  f1-score   support

       False       0.69      0.72      0.71      5067
        True       0.42      0.39      0.41      2640

    accuracy                           0.61      7707
   macro avg       0.56      0.56      0.56      7707
weighted avg       0.60      0.61      0.61      7707


Матрица ошибок: 
[[3668 1399]
 [1611 1029]]



### Создаем модель

In [ ]:
svm_lsvc = LinearSVC()

svm_lsvc.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

### Оцениваем модель

In [ ]:
# предсказываем результат на тестовой выборке
svm_lsvc_pred = svm_lsvc.predict(X_test)

print(f'Сравнение метрик: \n{classification_report(y_test, svm_lsvc_pred)}\n')
print(f'Матрица ошибок: \n{confusion_matrix(y_test, svm_lsvc_pred)}\n')

Сравнение метрик: 
              precision    recall  f1-score   support

       False       0.64      0.55      0.59      5067
        True       0.32      0.41      0.36      2640

    accuracy                           0.50      7707
   macro avg       0.48      0.48      0.48      7707
weighted avg       0.53      0.50      0.51      7707


Матрица ошибок: 
[[2799 2268]
 [1557 1083]]

